In [ ]:
# This is done by a script but this notebook was left as an example
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular
from msmrd2.integrators import overdampedLangevin as odLangevin
import msmrd2.tools.quaternions as quats
import msmrd2.tools.particleTools as particleTools
import msmrd2.tools.analysis as analysisTools
import timeit

In [ ]:
# Main parameters
numparticles = 2
boxsize = 6 #5 
D = 1.0
Drot = 1.0
relativeDistanceCutOff = 2.2
numTrajectories = 1000

In [ ]:
# Define integrator and boundary (over-damped Langevin)
dt = 0.0001 #0.00001 #0.000005
seed = -1 # Negative seed, uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevin(dt, seed, bodytype) 
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 100 #160 #200.0
angularStrength = 10 #20 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticleAngular = patchyParticleAngular(sigma, strength, angularStrength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticleAngular)

In [ ]:
# Generates FPT data from scratch
# Generates numTrajectories trajectories starting at a random position and ending in a given bound state.
# This yields the first passage times for a given bound state. Each trajectory is Integrated until 
# a bound state is reached. The output in the files is the elapsed time and final state.
traj = msmrd2.trajectories.patchyDimer(2,1) # dummy trajectory to calculate boundstate of patchydimer
outputBoundStates = [1, 2, 5, 6] # Inner bound
fptsA = []
fptsB = []
for i in range(numTrajectories):
    print("Simulation:", i, "   ", end="\r")
    partlist = particleTools.randomParticleList(numparticles, boxsize, relativeDistanceCutOff, D, Drot)
    unbound = True
    while(unbound):
        integrator.integrate(partlist)
        boundState = traj.getState(partlist[0], partlist[1])
        if ( (boundState >= 1) and (boundState <= 8)):
            if boundState in outputBoundStates:
                fptsA.append(integrator.clock)
                integrator.resetClock()
                unbound = False;
            else:
                fptsB.append(integrator.clock)
                integrator.resetClock()
                unbound = False;
                
        if integrator.clock >= 1500.0:  
            integrator.resetClock()
            unbound = False;

## Plots of benchmark FPT data

In [ ]:
hbins = range(0, 600,15)
plt.hist(fptsA, bins = hbins, alpha=0.5, label ='A');
plt.hist(fptsB, bins = hbins, alpha=0.5, label ='B');
plt.legend()

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples
numBootsrapSamples = 2000
mfptA, fptAstd = analysisTools.bootstrappingMFPTs(fptsA, numBootsrapSamples)
mfptB, fptBstd = analysisTools.bootstrappingMFPTs(fptsB, numBootsrapSamples)
print("Raw MFPTs (A and B): ", np.array(fptsA).mean(), np.array(fptsB).mean() )
print("Bootstrapping mean and std (A): ", mfptA, fptAstd)
print("Bootstrapping mean and std (B): ", mfptB, fptBstd)

## For testing:

In [ ]:
# Generates numTrajectories trajectories starting at a random position and ending in a given bound state.
# This yields the first passage times for a given bound state. Each trajectory is Integrated until 
# a bound state is reached. The output in the files is the elapsed time and final state.
traj = msmrd2.trajectories.patchyDimer(2,1) # dummy trajectory to calculate boundstate of patchydimer
outputBoundStates = [1, 2, 5, 6] # Inner bound 
for i in range(numTrajectories):
    partlist = randomPartList()
    unbound = True
    while(unbound):
        integrator.integrate(partlist)
        boundState = traj.getState(partlist[0], partlist[1])
        if boundState in outputBoundStates:
        #if ( (boundState >= 1) and (boundState <= 8)):
            print(integrator.clock, boundState)
            print(partlist[0].position, partlist[0].orientation)
            print(partlist[1].position, partlist[1].orientation)
            integrator.resetClock()
            break;

In [ ]:
# Create a vmd output to see if states obtained by getBoundState fucntion match those expected.
datafile  = open('../../data/vmd/tests/dimerStates.xyz', 'w')
datafile.write(str(3*len(partlist)) + '\n')
datafile.write(str(0) + '\n')
for j, part in enumerate(partlist):
    v0 = part.position
    v1 = v0 + 0.6*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
    v2 = v0 + 0.6*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
    if j == 0:
        stringtype = 'type_0'
    else:
        stringtype = 'type_1'
    datafile.write(stringtype + ' ' + ' '.join(map(str, v0)) + '\n')
    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
    datafile.write('type_3' + ' ' + ' '.join(map(str, v2)) + '\n')
msmrdvis.generateTCL_patchyProteinsV2(numparticles = numparticles, 
                                     outfname = "dimerStates", 
                                     tclfname = "../../data/vmd/tests/dimerStates2vmd.tcl")

In [ ]:
map(str, 5.5)